# Setting up the workspace

## Importing the necessary modules

In [ ]:
import sqlite3
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Setting working directory

In [ ]:
os.chdir('/Users/lewyiliang/Library/CloudStorage/OneDrive-SIM-SingaporeInstituteofManagement/Year 2/ST2195 - Programming for Data Science/Coursework/dataverse_files')
os.getcwd()  # Prints the current working directory

## Setting up the coursework_py database 

In [ ]:
os.remove("coursework_py.db")

In [ ]:
conn = sqlite3.connect('/Users/lewyiliang/Library/CloudStorage/OneDrive-SIM-SingaporeInstituteofManagement/Year 2/ST2195 - Programming for Data Science/Coursework/dataverse_files/coursework_py.db')

## Creating tables in the database

In [ ]:
# Loading files needed
airports = pd.read_csv("airports.csv", header=0)
carriers = pd.read_csv("carriers.csv", header=0)
planes = pd.read_csv("plane-data.csv", header=0)

planes = pd.read_csv("plane-data.csv", header=0)
ontime_2005 = pd.read_csv("2005.csv.bz2", header=0, compression='bz2')
ontime_2006 = pd.read_csv("2006.csv.bz2", header=0, compression='bz2')
ontime_2007 = pd.read_csv("2007.csv.bz2", header=0, compression='bz2')

In [ ]:
# Using concat method to merge 2005-2007 flight data into one variable
ontime_all = pd.concat([ontime_2005, ontime_2006, ontime_2007], ignore_index=True)
print(ontime_all.shape) # Dataset has 21735733 rows, 29 columns

## Importing data frames to coursework_r.db 

In [ ]:
airports.to_sql('airports', con = conn, if_exists='replace', index = False)

In [ ]:
carriers.to_sql('carriers', con = conn, if_exists='replace', index = False)

In [ ]:
planes.to_sql('planes', con = conn, if_exists='replace', index = False)

In [ ]:
ontime_all.to_sql('ontime', con = conn, if_exists='replace', index = False)

# Data Wrangling

In [ ]:
c = conn.cursor()

## Identifying anomalies in the "ontime" table

In [ ]:
ontime_DepTime_check = pd.read_sql_query("SELECT DepTime FROM ontime WHERE DepTime > 2359 ORDER BY DepTime DESC", conn)
print(ontime_DepTime_check.shape[0]) # 4250 DepTime > 2359

ontime_ArrTime_check = pd.read_sql_query("SELECT ArrTime FROM ontime WHERE ArrTime > 2359 ORDER BY ArrTime DESC", conn)
print(ontime_ArrTime_check.shape[0]) # 18406 ArrTime > 2359

ontime_CRSArrTime_check = pd.read_sql_query("SELECT CRSArrTime FROM ontime WHERE CRSArrTime > 2359 ORDER BY CRSArrTime DESC", conn)
print(ontime_CRSArrTime_check.shape[0]) # 726 CRSTime > 2359

## Dealing with the anomalies and updating the values in the table in the SQL database directly

In [ ]:
c.execute("UPDATE ontime SET DepTime = DepTime - 2400 WHERE DepTime > 2359")
print(c.rowcount) # 4250 records updated

c.execute("UPDATE ontime SET ArrTime = ArrTime - 2400 WHERE ArrTime > 2359")
print(c.rowcount) # 18406 records updated

c.execute("UPDATE ontime SET CRSArrTime = CRSArrTime - 2400 WHERE CRSArrTime > 2359")
print(c.rowcount) # 726 records updated

# Q1. When is the best time of day, day of the week, and time of year to fly to minimise delays?

## Part 1: Best time of day to fly to minimise delays

### Categorizing flights in the 'ontime' table by the scheduled departure time by intervals of 2 hours

In [ ]:
# Adding a new column named "DepTimeInterval"
c.execute("ALTER TABLE ontime ADD DepTimeInterval")
conn.commit()

In [ ]:
# Grouping the respective departure times to the corresponding departure time interval (in 2-hour intervals)
c.execute("""
          UPDATE ontime 
          SET DepTimeInterval = CASE
            WHEN CRSDepTime BETWEEN 0 AND 199 THEN '00.00-01.59' 
            WHEN CRSDepTime BETWEEN 200 AND 399 THEN '02.00-03.59' 
            WHEN CRSDepTime BETWEEN 400 AND 599 THEN '04.00-05.59' 
            WHEN CRSDepTime BETWEEN 600 AND 799 THEN '06.00-07.59' 
            WHEN CRSDepTime BETWEEN 800 AND 999 THEN '08.00-09.59' 
            WHEN CRSDepTime BETWEEN 1000 AND 1199 THEN '10.00-11.59' 
            WHEN CRSDepTime BETWEEN 1200 AND 1399 THEN '12.00-13.59' 
            WHEN CRSDepTime BETWEEN 1400 AND 1599 THEN '14.00-15.59' 
            WHEN CRSDepTime BETWEEN 1600 AND 1799 THEN '16.00-17.59' 
            WHEN CRSDepTime BETWEEN 1800 AND 1999 THEN '18.00-19.59' 
            WHEN CRSDepTime BETWEEN 2000 AND 2199 THEN '20.00-21.59' 
            WHEN CRSDepTime BETWEEN 2200 AND 2399 THEN '22.00-23.59' 
            ELSE NULL
          END
          """)
conn.commit()
print(c.rowcount) # Dataset has 21735733 rows, thus the row count should tally

#### Looking at the distribution of flights for each departure time interval

In [ ]:
# Distribution of flights for each departure time interval
interval_overview = pd.read_sql_query("""
                                      SELECT DepTimeInterval, COUNT(*) AS NumOfFlights 
                                      FROM ontime
                                      GROUP BY DepTimeInterval""", conn)

In [ ]:
display(interval_overview)

In [ ]:
# Obtaining the average arrival delay for each departure time interval
q1p1a = pd.read_sql("""
                    SELECT DepTimeInterval, AVG(ArrDelay) AS AvgArrDelay 
                    FROM ontime 
                    WHERE Cancelled = 0 AND Diverted = 0 
                    GROUP BY DepTimeInterval 
                    ORDER BY DepTimeInterval
                    """, conn)

In [ ]:
# Changing the column names of q1p1a
q1p1a.columns = ["Departure Time Interval", "Average Arrival Delay"]

# Printing the output of q1p1a
display(q1p1a)

In [ ]:
# Plot of average arrival delay of flights by departure time interval
fig, ax = plt.subplots()
q1p1a.plot(kind="bar", x="Departure Time Interval", y="Average Arrival Delay", color="#90d2d8", ax=ax, width=0.8, edgecolor="black", legend=None)
q1p1a.plot(x="Departure Time Interval", y="Average Arrival Delay", color="black", ax=ax, legend=None)
q1p1a.plot(kind="scatter", x="Departure Time Interval", y="Average Arrival Delay", color="#e1ff00", s=25, edgecolor="black", ax=ax)
ax.set(title="Figure 1.1a: Average Arrival Delay of Flights", xlabel="Departure Time Interval (24hr)", ylabel="Average Arrival Delay (in mins)")
ax.set_title("Figure 1.1a: Average Arrival Delay of Flights", fontweight="bold")
ax.set_ylim(-0.15, 18)
plt.xticks(rotation=20, ha="right")

for i, r in q1p1a.iterrows():
    ax.text(i, r["Average Arrival Delay"] + 0.5, round(r["Average Arrival Delay"], 1), ha="center", fontsize=8)

In [ ]:
# To obtain the percentage of cancelled, diverted, delayed and on-time performance for flights
q1p1b = pd.read_sql("""
                    SELECT DepTimeInterval, (AVG(Cancelled)*100) AS AvgCnl_perc, (AVG(Diverted)*100) AS AvgDiv_perc, 
                        ((CAST(SUM(CASE WHEN ArrDelay > 0 THEN 1 ELSE 0 END) AS FLOAT)/CAST(COUNT(*) AS FLOAT))*100) AS AvgArrDelay_perc, 
                        ((CAST(SUM(CASE WHEN ArrDelay <= 15 THEN 1 ELSE 0 END) AS FLOAT)/CAST(COUNT(*) AS FLOAT))*100) AS on_time_performance 
                    FROM ontime 
                    GROUP BY DepTimeInterval 
                    ORDER BY DepTimeInterval""", conn)

In [ ]:
# Changing the column names of q1p1b
q1p1b.columns = ["Departure Time Interval", "% of Cancelled Flights", "% of Diverted Flights", "% of Delayed Flights", "On-Time Performance in %"]

# Printing the output of q1p1b
display(q1p1b)

In [ ]:
# Bar chart for percentage of cancelled flights
fig, ax = plt.subplots()
q1p1b.plot(kind="bar", x="Departure Time Interval", y="% of Cancelled Flights", color="#ffecb8", ax=ax, width=0.8, edgecolor="black", legend=None)
q1p1b.plot(x="Departure Time Interval", y="% of Cancelled Flights", color="black", ax=ax, legend=None)
q1p1b.plot(kind="scatter", x="Departure Time Interval", y="% of Cancelled Flights", color="#e1ff00", s=25, edgecolor="black", ax=ax)
ax.set(title="Figure 1.1b: % of Cancelled Flights by Time of Day", xlabel="Departure Time Interval (24hr)", ylabel="Cancelled Flights (in %)")
ax.set_title("Figure 1.1b: % of Cancelled Flights by Time of Day", fontweight="bold")
ax.set_ylim(0, 2.75)
plt.xticks(rotation=20, ha="right")

for i, r in q1p1b.iterrows():
    ax.text(i, r["% of Cancelled Flights"] + 0.05, f"{round(r['% of Cancelled Flights'], 2)}%", ha="center", fontsize=8)

In [ ]:
# Bar chart for percentage of delayed flights
fig, ax = plt.subplots()
ax.bar(x=q1p1b['Departure Time Interval'], height=q1p1b['% of Delayed Flights'], color="#f6a6b2", width=0.8, edgecolor="black")
ax.plot(q1p1b['Departure Time Interval'], q1p1b['% of Delayed Flights'], color="black")
ax.scatter(x=q1p1b['Departure Time Interval'], y=q1p1b['% of Delayed Flights'], color="#e1ff00", s=25, edgecolor="black")
ax.set(title="Figure 1.1c: % of Delayed Flights by Time of Day", xlabel="Departure Time Interval (24hr)", ylabel="Delayed Flights (in %)")
ax.set_title("Figure 1.1c: % of Delayed Flights by Time of Day", fontweight="bold")
ax.set_ylim(0, 57.23)
plt.xticks(rotation=20, ha="right")

for i, r in q1p1b.iterrows():
    ax.text(i, r["% of Delayed Flights"] + 0.5, f"{round(r['% of Delayed Flights'], 2)}%", ha="center", fontsize=8)

In [ ]:
# Bar chart for on-time performance of flights in percentage
    # On-time performance measures punctuality within 15 minutes of the scheduled arrival time
fig, ax = plt.subplots()
ax.bar(x=q1p1b['Departure Time Interval'], height=q1p1b['On-Time Performance in %'], color="#b7ded2", width=0.8, edgecolor="black")
ax.plot(q1p1b['Departure Time Interval'], q1p1b['On-Time Performance in %'], color="black")
ax.scatter(x=q1p1b['Departure Time Interval'], y=q1p1b['On-Time Performance in %'], color="#e1ff00", s=25, edgecolor="black")
ax.set(title="Figure 1.1d: On-time Performance (%) by Time of Day", xlabel="Departure Time Interval (24hr)", ylabel="On-time Performance (in %)")
ax.set_title("Figure 1.1d: On-time Performance (%) by Time of Day", fontweight="bold")
ax.set_ylim(0, 95.4)
plt.xticks(rotation=20, ha="right")

for i, r in q1p1b.iterrows():
    ax.text(i, r["On-Time Performance in %"] + 1, f"{round(r['On-Time Performance in %'], 2)}%", ha="center", fontsize=8)

In [ ]:
# Bar chart for percentage of diverted flights 
    # Note: This chart was not included in the report as values were negligible
fig, ax = plt.subplots()
q1p1b.plot(kind="bar", x="Departure Time Interval", y="% of Diverted Flights", color="#f7c297", ax=ax, width=0.8, edgecolor="black", legend=None)
q1p1b.plot(x="Departure Time Interval", y="% of Diverted Flights", color="black", ax=ax, legend=None)
q1p1b.plot(kind="scatter", x="Departure Time Interval", y="% of Diverted Flights", color="#e1ff00", s=25, edgecolor="black", ax=ax)
ax.set(title="Figure 1.1e: % of Diverted Flights by Time of Day", xlabel="Departure Time Interval (24hr)", ylabel="Diverted Flights (in %)")
ax.set_title("Figure 1.1e: % of Diverted Flights by Time of Day", fontweight="bold")
ax.set_ylim(0, 0.286)
plt.xticks(rotation=20, ha="right")

for i, r in q1p1b.iterrows():
    ax.text(i, r["% of Diverted Flights"] + 0.005, f"{round(r['% of Diverted Flights'], 2)}%", ha="center", fontsize=8)

## Part 2: Best day of week to fly to minimise delays

### Categorizing flights in the 'ontime' table by the seasons

In [ ]:
# Adding a new column in the "ontime" table named "Season"
c.execute("ALTER TABLE ontime ADD Season")
conn.commit()

In [ ]:
# Grouping the respective months to the corresponding season
c.execute("""
    UPDATE ontime 
    SET Season = CASE 
        WHEN Month BETWEEN 3 AND 5 THEN 'Spring'
        WHEN Month BETWEEN 6 AND 8 THEN 'Summer'
        WHEN Month BETWEEN 9 AND 11 THEN 'Autumn'
        ELSE 'Winter'
    END
""")
conn.commit()

In [ ]:
# Obtaining the average arrival delay for each day of week and season
q1p2a = pd.read_sql_query("""
                          SELECT Season, DayOfWeek, AVG(ArrDelay) AS AvgArrDelay 
                          FROM ontime 
                          WHERE Cancelled = 0 AND Diverted = 0 
                          GROUP BY Season, DayOfWeek 
                          ORDER BY AvgArrDelay
                          """, conn)

In [ ]:
# Changing the column names of q1p2a
q1p2a.columns = ["Season", "Day of Week", "Average Arrival Delay"]

# Converting 'Season' column to categorical data type
q1p2a['Season'] = pd.Categorical(q1p2a['Season'], categories=['Spring', 'Summer', 'Autumn', 'Winter'])

# Printing the output of q1p2a
display(q1p2a)

In [ ]:
# Plot of average arrival delay of flights by day of week and season
g = sns.catplot(x='Day of Week', y='Average Arrival Delay', hue='Season', kind='bar', data=q1p2a, 
                palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'},
                height=4, aspect=1.5, col='Season', col_order=['Spring', 'Summer', 'Autumn', 'Winter'],
                sharey=False)
g.despine(left=True)
g.set_axis_labels("Day of Week", "Average Arrival Delay (in mins)")
g.fig.suptitle('Figure 1.2a: Average Arrival Delay by Day of Week and Season', fontsize=16, fontweight='bold', y=1.05)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)

# add horizontal dashed lines for average delay for each season
for ax, (season, data) in zip(g.axes.flat, q1p2a.groupby('Season')):
    avg_delay = data['Average Arrival Delay'].mean()
    ax.axhline(y=avg_delay, linestyle='--', color='red')

# set x-tick labels
g.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

plt.show()

In [ ]:
# To obtain the percentage of cancelled, diverted, delayed and on-time performance for flights
q1p2b = pd.read_sql_query("""
                          SELECT Season, DayOfWeek, (AVG(Cancelled)*100) AS AvgCnl_perc, (AVG(Diverted)*100) AS AvgDiv_perc, 
                            ((CAST(SUM(CASE WHEN ArrDelay > 0 THEN 1 ELSE 0 END) AS FLOAT)/CAST(COUNT(*) AS FLOAT))*100) AS AvgArrDelay_perc, 
                            ((CAST(SUM(CASE WHEN ArrDelay <= 15 THEN 1 ELSE 0 END) AS FLOAT)/CAST(COUNT(*) AS FLOAT))*100) AS on_time_performance 
                          FROM ontime 
                          GROUP BY Season, DayOfWeek 
                          ORDER BY DayOfWeek, Season
                          """, conn)

In [ ]:
# Changing the column names of q1p2b
q1p2b.columns = ["Season", "Day of Week", "% of Cancelled Flights", "% of Diverted Flights", "% of Delayed Flights", "On-time Performance in %"]

# Converting 'Season' column to categorical data type
q1p2b["Season"] = pd.Categorical(q1p2b["Season"], categories=["Spring", "Summer", "Autumn", "Winter"])

# Printing the output of q1p2b
display(q1p2b)

In [ ]:
# Bar chart for percentage of cancelled flights 
g = sns.catplot(x='Day of Week', y='% of Cancelled Flights', hue='Season', kind='bar', data=q1p2b, 
                palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'},
                height=4, aspect=1.5, col='Season', col_order=['Spring', 'Summer', 'Autumn', 'Winter'],
                sharey=False)
g.despine(left=True)
g.set_axis_labels("Day of Week", "Cancelled Flights (in %)")
g.fig.suptitle('Figure 1.2b: Cancelled Flights (in %) by Day of Week and Season', fontsize=16, fontweight='bold', y=1.05)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)

# add horizontal dashed lines for average delay for each season
for ax, (season, data) in zip(g.axes.flat, q1p2b.groupby('Season')):
    avg_delay = data['% of Cancelled Flights'].mean()
    ax.axhline(y=avg_delay, linestyle='--', color='red')

# set x-tick labels
g.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

plt.show()

In [ ]:
# Bar chart for percentage of delayed flights 
g = sns.catplot(x='Day of Week', y='% of Delayed Flights', hue='Season', kind='bar', data=q1p2b, 
                palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'},
                height=4, aspect=1.5, col='Season', col_order=['Spring', 'Summer', 'Autumn', 'Winter'],
                sharey=False)
g.despine(left=True)
g.set_axis_labels("Day of Week", "Delayed Flights (in %)")
g.fig.suptitle('Figure 1.2c: Delayed Flights (in %) by Day of Week and Season', fontsize=16, fontweight='bold', y=1.05)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)

# add horizontal dashed lines for average delay for each season
for ax, (season, data) in zip(g.axes.flat, q1p2b.groupby('Season')):
    avg_delay = data['% of Delayed Flights'].mean()
    ax.axhline(y=avg_delay, linestyle='--', color='red')

# set x-tick labels
g.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

plt.show()

In [ ]:
# Bar chart for on-time performance of flights in percentage
g = sns.catplot(x='Day of Week', y='On-time Performance in %', hue='Season', kind='bar', data=q1p2b, 
                palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'},
                height=4, aspect=1.5, col='Season', col_order=['Spring', 'Summer', 'Autumn', 'Winter'],
                sharey=False)
g.despine(left=True)
g.set_axis_labels("Day of Week", "On-time Performance (in %)")
g.fig.suptitle('Figure 1.2d: On-time Performance (in %) by Day of Week and Season', fontsize=16, fontweight='bold', y=1.05)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)

# add horizontal dashed lines for average delay for each season
for ax, (season, data) in zip(g.axes.flat, q1p2b.groupby('Season')):
    avg_delay = data['On-time Performance in %'].mean()
    ax.axhline(y=avg_delay, linestyle='--', color='red')

# set x-tick labels
g.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

plt.show()

In [ ]:
# Bar chart for percentage of diverted flights 
g = sns.catplot(x='Day of Week', y='% of Diverted Flights', hue='Season', kind='bar', data=q1p2b, 
                palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'},
                height=4, aspect=1.5, col='Season', col_order=['Spring', 'Summer', 'Autumn', 'Winter'],
                sharey=False)
g.despine(left=True)
g.set_axis_labels("Day of Week", "Diverted Flights (in %)")
g.fig.suptitle('Figure 1.2e: Diverted Flights (in %) by Day of Week and Season', fontsize=16, fontweight='bold', y=1.05)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)

# add horizontal dashed lines for average delay for each season
for ax, (season, data) in zip(g.axes.flat, q1p2b.groupby('Season')):
    avg_delay = data['% of Diverted Flights'].mean()
    ax.axhline(y=avg_delay, linestyle='--', color='red')

# set x-tick labels
g.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

plt.show()

### Part 3: Best time of year to fly to minimise delays

In [ ]:
# Obtaining the average arrival delay for each month and season
q1p3a = pd.read_sql_query("""
                          SELECT Season, Month, AVG(ArrDelay) AS AvgArrDelay 
                          FROM ontime 
                          WHERE Cancelled = 0 AND Diverted = 0 
                          GROUP BY Season, Month 
                          ORDER BY Month, Season
                          """, conn)

In [ ]:
# Changing the column names of q1p3b
q1p3a.columns = ["Season", "Month", "Average Arrival Delay"]
q1p3a["Season"] = pd.Categorical(q1p3a["Season"], categories=["Spring", "Summer", "Autumn", "Winter"])

# Printing the output of q1p3b
display(q1p3a)

In [ ]:
# Plot of average arrival delay of flights by month and season
plt.figure(figsize=(10, 6))
sns.set_style('whitegrid')

barplot = sns.barplot(data=q1p3a, x='Month', y='Average Arrival Delay', hue='Season', palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'}, order=range(1, 13), dodge=False)
plt.title('Figure 1.3a: Average Arrival Delay by Month and Season', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Average Arrival Delay (minutes)')
plt.xticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.ylim(bottom=0, top=15)
plt.axhline(y=q1p3a['Average Arrival Delay'].mean(), ls='--', color='r', label=f"Overall Mean: {q1p3a['Average Arrival Delay'].mean():.2f}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=5)
plt.tight_layout()

# Adding text labels to the plot
ax = barplot.axes
for i, r in q1p3a.iterrows():
    ax.text(i, r["Average Arrival Delay"] + 0.2, f"{round(r['Average Arrival Delay'], 1)}%", ha="center", fontsize=10)

plt.show()

In [ ]:
# To obtain the percentage of cancelled, diverted, delayed and on-time performance for flights
q1p3b = pd.read_sql_query("""
                          SELECT Season, Month, (AVG(Cancelled)*100) AS AvgCnl_perc, (AVG(Diverted)*100) AS AvgDiv_perc, 
                            ((CAST(SUM(CASE WHEN ArrDelay > 0 THEN 1 ELSE 0 END) AS FLOAT)/CAST(COUNT(*) AS FLOAT))*100) AS AvgArrDelay_perc, 
                            ((CAST(SUM(CASE WHEN ArrDelay <= 15 THEN 1 ELSE 0 END) AS FLOAT)/CAST(COUNT(*) AS FLOAT))*100) AS on_time_performance 
                          FROM ontime 
                          GROUP BY Season, Month 
                          ORDER BY Month
                          """, conn)

In [ ]:
# Changing the column names of q1p3b
q1p3b.columns = ["Season", "Month", "% of Cancelled Flights", "% of Diverted Flights", "% of Delayed Flights", "On-time Performance in %"]
q1p3b["Season"] = pd.Categorical(q1p3b["Season"], categories=["Spring", "Summer", "Autumn", "Winter"])

# Printing the output of q1p3b
display(q1p3b)

In [ ]:
# Bar chart for percentage of cancelled flights 
barplot = sns.barplot(data=q1p3b, x='Month', y='% of Cancelled Flights', hue='Season', palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'}, order=range(1, 13), dodge=False)
plt.title('Figure 1.3b: Percentage of Cancelled Flights by Month and Season', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Cancelled Flights (in %)')
plt.xticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.ylim(bottom=0, top=3.5)
plt.axhline(y=q1p3b['% of Cancelled Flights'].mean(), ls='--', color='r', label=f"Overall Mean: {q1p3b['% of Cancelled Flights'].mean():.2f}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)
plt.tight_layout()

# add text labels to the plot
ax = barplot.axes
for i, r in q1p3b.iterrows():
    ax.text(i, r["% of Cancelled Flights"] + 0.05, f"{round(r['% of Cancelled Flights'], 2)}%", ha="center", fontsize=8)

plt.show()

In [ ]:
# Bar chart for percentage of delayed flights 
barplot = sns.barplot(data=q1p3b, x='Month', y='% of Delayed Flights', hue='Season', palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'}, order=range(1, 13), dodge=False)
plt.title('Figure 1.3c: Percentage of Delayed Flights by Month and Season', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Delayed Flights (in %)')
plt.xticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.ylim(bottom=0, top=55)
plt.axhline(y=q1p3b['% of Delayed Flights'].mean(), ls='--', color='r', label=f"Overall Mean: {q1p3b['% of Delayed Flights'].mean():.2f}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)
plt.tight_layout()

# add text labels to the plot
ax = barplot.axes
for i, r in q1p3b.iterrows():
    ax.text(i, r["% of Delayed Flights"] + 1, f"{round(r['% of Delayed Flights'], 1)}%", ha="center", fontsize=8)

plt.show()

In [ ]:
# Bar chart for on-time performance of flights in percentage
barplot = sns.barplot(data=q1p3b, x='Month', y='On-time Performance in %', hue='Season', palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'}, order=range(1, 13), dodge=False)
plt.title('Figure 1.3d: On-time Performance (%) by Month and Season', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Cancelled Flights (in %)')
plt.xticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.ylim(bottom=0, top=90)
plt.axhline(y=q1p3b['On-time Performance in %'].mean(), ls='--', color='r', label=f"Overall Mean: {q1p3b['On-time Performance in %'].mean():.2f}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)
plt.tight_layout()

# add text labels to the plot
ax = barplot.axes
for i, r in q1p3b.iterrows():
    ax.text(i, r["On-time Performance in %"] + 1.3, f"{round(r['On-time Performance in %'], 1)}%", ha="center", fontsize=8)

plt.show()

In [ ]:
barplot = sns.barplot(data=q1p3b, x='Month', y='% of Diverted Flights', hue='Season', palette={'Spring': '#00FF7F', 'Summer': '#FFD700', 'Autumn': '#FFA500', 'Winter': '#87CEEB'}, order=range(1, 13), dodge=False)
plt.title('Figure 1.3e: Percentage of Diverted Flights by Month and Season', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Diverted Flights (in %)')
plt.xticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.ylim(bottom=0, top=0.5)
plt.axhline(y=q1p3b['% of Diverted Flights'].mean(), ls='--', color='r', label=f"Overall Mean: {q1p3b['% of Diverted Flights'].mean():.2f}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)
plt.tight_layout()

# add text labels to the plot
ax = barplot.axes
for i, r in q1p3b.iterrows():
    ax.text(i, r["% of Diverted Flights"] + 0.005, f"{round(r['% of Diverted Flights'], 1)}%", ha="center", fontsize=8)

plt.show()

## Close the database connection

In [ ]:
conn.close()